# Hand sign prediction using MobileNet

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import os
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import random

In [2]:
mobileNet = tf.keras.applications.mobilenet.MobileNet()

In [3]:
os.chdir('datasets/Sign-Language-Digits-Dataset')

In [4]:
os.getcwd()

'/home/google/Documents/Deep Learning Implemantation/CNN/datasets/Sign-Language-Digits-Dataset'

In [5]:
os.listdir()

['test', 'train', 'valid']

In [24]:
# Creating train,test and valid directory

if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('test')
    os.mkdir('valid')
    
    for i in range(0, 10):   # Here iterate over all the directories(0-9). i represent directories
        shutil.move(f'{i}', 'train')  # Here we move each directory to the train directory
        os.mkdir(f'valid/{i}') # Here we just create directories for "0" to '9' one at a time in valid directory
        os.mkdir(f'test/{i}')  # Here we just create directories for "0" to '9' one at a time in test directory

        valid_samples = random.sample(os.listdir(f'train/{i}'), 30)  # Here we took 30 samples from each dir inside train dir.
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}') # Here we just move 30 samples(represented by 'j' here) from train to valid dir.

        test_samples = random.sample(os.listdir(f'train/{i}'), 5)  # Here we took 5 samples from each dir inside train dir.
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}') # Here we just move 5 samples(represented by 'k' here) from train to test dir.
    os.chdir('../..')

In [7]:
os.chdir('../..')

In [8]:
os.getcwd()

'/home/google/Documents/Deep Learning Implemantation/CNN'

In [9]:
train_path = 'datasets/Sign-Language-Digits-Dataset/train'

In [10]:
test_path = 'datasets/Sign-Language-Digits-Dataset/test'
valid_path = 'datasets/Sign-Language-Digits-Dataset/valid'

Preprocessing our data for MobileNet model

In [11]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(directory= test_path, target_size=(224,224),batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) \
.flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)

Found 1712 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


**Modifying our our MobileNet Model**

In [12]:
X = mobileNet.layers[-6].output

In [15]:
output = Dense(10, activation = 'softmax')(X)

This gonna be our output layer because we have 10 classes to o/p

In [18]:
inputs = mobileNet.input

In [20]:
model = Model(inputs,output) # Here  we pass the mobileNet inputs to our model inputs and our dense layer for output

In [21]:
# Lets freez some layer for better result 
for layer in model.layers[:-23]:
    layer.trainable = False
# Here we freez all the layers except last 23 layers out of 81 layers

In [22]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

**Now let's train our model**

In [23]:
model.compile(optimizer=Adam(learning_rate=0.001),loss = 'categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(x = train_batches,validation_data = valid_batches,epochs=10,verbose = 2)

Epoch 1/10
172/172 - 69s - loss: 0.3965 - accuracy: 0.8633 - val_loss: 0.8745 - val_accuracy: 0.7600
Epoch 2/10
172/172 - 67s - loss: 0.0903 - accuracy: 0.9784 - val_loss: 0.0768 - val_accuracy: 0.9767
Epoch 3/10
172/172 - 67s - loss: 0.0536 - accuracy: 0.9831 - val_loss: 0.1892 - val_accuracy: 0.9400
Epoch 4/10
172/172 - 67s - loss: 0.0471 - accuracy: 0.9842 - val_loss: 0.2039 - val_accuracy: 0.9333
Epoch 5/10
172/172 - 66s - loss: 0.0206 - accuracy: 0.9930 - val_loss: 0.3079 - val_accuracy: 0.8967
Epoch 6/10
172/172 - 65s - loss: 0.0187 - accuracy: 0.9942 - val_loss: 0.0422 - val_accuracy: 0.9967
Epoch 7/10
172/172 - 65s - loss: 0.0244 - accuracy: 0.9924 - val_loss: 0.0584 - val_accuracy: 0.9900
Epoch 8/10
172/172 - 65s - loss: 0.0777 - accuracy: 0.9784 - val_loss: 0.2569 - val_accuracy: 0.9200
Epoch 9/10
172/172 - 66s - loss: 0.0266 - accuracy: 0.9901 - val_loss: 0.0171 - val_accuracy: 0.9967
Epoch 10/10
172/172 - 66s - loss: 0.0215 - accuracy: 0.9924 - val_loss: 0.3689 - val_accura

**Now its prediction Time Guys!!!**

In [25]:
pred = model.predict(x = test_batches, verbose = 0)

In [27]:
from sklearn.metrics import classification_report,confusion_matrix

In [30]:
print(confusion_matrix(y_true = test_batches.classes, y_pred = pred.argmax(axis = -1)))

[[1 1 0 0 1 1 0 0 1 0]
 [0 1 0 0 0 0 0 1 3 0]
 [1 0 0 0 1 1 0 0 1 1]
 [0 0 0 0 0 0 3 0 2 0]
 [0 0 2 1 0 0 1 0 1 0]
 [0 1 1 0 1 0 0 0 1 1]
 [0 0 1 1 1 0 0 0 1 1]
 [0 0 1 1 0 2 0 0 1 0]
 [1 2 0 1 0 0 1 0 0 0]
 [2 0 0 1 1 1 0 0 0 0]]


# Great Job Dude!!